In [1]:
from __future__ import print_function
import numpy as np

from hyperopt import Trials, STATUS_OK, tpe
from keras.layers.core import Dense, Dropout, Activation
from keras.models import Sequential
from keras.utils import np_utils

from hyperas import optim
from hyperas.distributions import choice, uniform



Using TensorFlow backend.


In [2]:
from keras.applications.resnet50 import ResNet50
import numpy as np
import re
from scipy.io import loadmat
import os
from keras.preprocessing import image 
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from keras import models
from keras import layers
from keras import optimizers
from keras import callbacks
import keras.backend as K
import tensorflow as tf
import math
from keras.callbacks import LearningRateScheduler
from keras import regularizers
import matplotlib.pyplot as plt

In [3]:
def data():
    base_dir="/home/ankur248/Documents/Europe/Fashion144k_v1/photos/"
    conv_base = ResNet50(weights='imagenet',include_top=False,pooling='avg',input_shape=(378,256,3))
    
    relvotes = loadmat('/home/ankur248/Documents/Europe/Fashion144k_v1/feat/relvotes.mat')
    relvotes=np.array(relvotes['X'])
    
    f=open("/home/ankur248/Documents/Europe/Fashion144k_v1/photos.txt", "r")

    if f.mode == 'r':
        contents =f.read()
            
    image_order=[]          
    image_order.append(re.findall(r"[-+]?\d*\.\d+|\d+", contents))
    image_order=np.array(image_order)
    
    order=contents.split('\n')
    order=np.array(order)
    
    train_split=order[0:3000]
    train_labels=relvotes[0][0:3000]
    validation_split=order[3000:4000]
    validation_labels=relvotes[0][3000:4000]
    test_split=order[4000:5000]
    test_labels=relvotes[0][4000:5000]
    
    encoder = LabelEncoder()
    encoder.fit(train_labels)
    encoded_train_labels = encoder.transform(train_labels)
    encoded_train_labels = np_utils.to_categorical(encoded_train_labels)

    encoder.fit(validation_labels)
    encoded_validation_labels = encoder.transform(validation_labels)
    encoded_validation_labels = np_utils.to_categorical(encoded_validation_labels)

    encoder.fit(test_labels)
    encoded_test_labels = encoder.transform(test_labels)
    encoded_test_labels = np_utils.to_categorical(encoded_test_labels)
    
    train_images=[]
    x=[]
    i=0
    while(i<len(train_split)):
        x.append(base_dir+str(train_split[i]))
        img=image.load_img(base_dir+str(train_split[i]),target_size=(378,256))
        img=image.img_to_array(img)
        train_images.append(img)
        i=i+1
    train_images=np.array(train_images)
    
    validation_images=[]
    i=0
    y=[]
    while(i<len(validation_split)):
        y.append(base_dir+str(validation_split[i]))
        img=image.load_img(base_dir+str(validation_split[i]),target_size=(378,256))
        img=image.img_to_array(img)
        validation_images.append(img)
        i=i+1
    validation_images=np.array(validation_images)
    return train_images,encoded_train_labels,validation_images,encoded_validation_labels

In [4]:
def create_model(x_train, y_train, x_test, y_test):
    def step_decay(epoch):
        gamma=0.001
        power=-0.75
        baselr=1e-5
        alpha=baselr*(1+(gamma*epoch*5))**(power)
        return float(alpha)
    
    def my_plot(h):  
        mae = h['acc']
        val_mae = h['val_acc']
        loss = h['loss']
        val_loss = h['val_loss']
        epochs = range(1,len(mae)+1)

        plt.plot(epochs,mae,'r',label='Training acc')
        plt.plot(epochs,val_mae,'b',label='Validation acc')
        plt.title('Training and validation acc')
        plt.legend()

        plt.figure()
        plt.plot(epochs,loss,'r',label='Training loss')
        plt.plot(epochs,val_loss,'b',label='Validation loss')
        plt.title('Training and validation loss')
        plt.legend()
    
        plt.show()
    
    callbacks_list = [
    callbacks.ModelCheckpoint(filepath='Human.h5',monitor='val_loss',save_best_only=True,),
    LearningRateScheduler(step_decay)
    ] 

    model = models.Sequential()

    model.add(conv_base)
    
    model.add(layers.Dropout(0.85))
    model.add(layers.Dense(10,activation='softmax',input_dim=2048,kernel_regularizer=regularizers.l2()))


    model.compile(optimizer=,
             loss='categorical_crossentropy',
             metrics=['accuracy'])
    result= model.fit(train_images,encoded_train_labels,
                  epochs=4,
                  batch_size=16, 
                  callbacks=callbacks_list,
                  validation_data=(validation_images,encoded_validation_labels))
    my_plot(result.history)
    val_acc = np.amax(result.history['val_acc']) 
    print('Best val_acc of epoch:', val_acc)
    return {'loss': val_acc, 'status': STATUS_OK, 'model': model}


In [5]:
best_run, best_model = optim.minimize(model=create_model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=5,
                                          trials=Trials(),
                                     notebook_name='Hyperas-Human_Centered_Geolocation')


>>> Imports:
#coding=utf-8

from __future__ import print_function

try:
    import numpy as np
except:
    pass

try:
    from hyperopt import Trials, STATUS_OK, tpe
except:
    pass

try:
    from keras.layers.core import Dense, Dropout, Activation
except:
    pass

try:
    from keras.models import Sequential
except:
    pass

try:
    from keras.utils import np_utils
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
    from hyperas.distributions import choice, uniform
except:
    pass

try:
    from keras.applications.resnet50 import ResNet50
except:
    pass

try:
    import numpy as np
except:
    pass

try:
    import re
except:
    pass

try:
    from scipy.io import loadmat
except:
    pass

try:
    import os
except:
    pass

try:
    from keras.preprocessing import image
except:
    pass

try:
    from sklearn.preprocessing import LabelEncoder
except:
    pass

try:
    from keras.utils import np_utils
except:
    pass

try:
    from keras.wrappers

ResourceExhaustedError: OOM when allocating tensor with shape[1,1,1024,2048] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node res5a_branch1/truncated_normal/TruncatedNormal (defined at /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4185)  = TruncatedNormal[T=DT_INT32, dtype=DT_FLOAT, seed=87654321, seed2=1409137, _device="/job:localhost/replica:0/task:0/device:GPU:0"](res5a_branch1/truncated_normal/shape)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Caused by op 'res5a_branch1/truncated_normal/TruncatedNormal', defined at:
  File "/usr/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.

In [ ]:
X_train, Y_train, X_test, Y_test = data()
print("Evalutation of best performing model:")
print(best_model.evaluate(X_test, Y_test))
print("Best performing model chosen hyper-parameters:")
print(best_run)